# EC2 - shrinkage

In [18]:
%matplotlib inline
import math as m
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from ipywidgets import interact

In [19]:
h0 = widgets.FloatSlider(value=200,min=100,max=1000,step=0.025,description='h0 (mm):',readout_format='.0f')
t = widgets.FloatLogSlider(value=100000,min=0,max=6,step=0.025,description='t (d):',readout_format='.0f')
ts = widgets.FloatSlider(value=3,min=1,max=10,step=1,description='ts (d):',readout_format='.0f')
rh = widgets.FloatSlider(value=80,min=0,max=100,step=1,description='RH (%):',readout_format='.0f')
fck = widgets.FloatSlider(value=20,min=20,max=50,step=5,description='fck (MPa):',readout_format='.0f')
fck2 = widgets.Dropdown(
    options=[('C20/25', 20.0), ('C25/30', 25.0), ('C30/37', 30.0), ('C35/45', 35.0), ('C40/50', 40.0), ('C45/55', 45.0), ('C50/60', 50.0)],
    value=20,
    description='Concrete:',
)
cem = widgets.Dropdown(
    options=['Type R', 'Type N', 'Type S'],
    value='Type N',
    description='Cement:'
)
cemprops = {
    'Type S': [3, 0.13],
    'Type N': [4, 0.12],
    'Type R': [6, 0.11],
}

In [20]:
def shrink_strain(t=28, h0=100, ts=3, rh=65, fck=20.0, cem='Type N'):
    fcm = fck+8
    alpha1 = cemprops[cem][0]
    alpha2 = cemprops[cem][1]

    eps_ca = 25.0e-6*(fck-10)
    beta_as = 1.0-m.exp(-0.2*(t**0.5))

    beta_rh = 1.55*(1.0-(rh/100)**3)
    eps_cd = beta_rh*0.85e-6*((220+110*alpha1)*m.exp(-alpha2*fcm/10.0))
    beta_ds = (t-ts)/((t-ts)+0.4*h0**1.5)
    
    return beta_as*eps_ca + beta_ds*eps_cd
    
@interact(h0=h0, t=t, ts=ts, rh=rh, fck=fck, cem=cem)
def shrinkstrain(t=28, h0=100, ts=3, rh=65, fck=20.0, cem=0.0):
    shrink = shrink_strain(t=t, h0=h0, ts=ts, rh=rh, fck=fck, cem=cem)
    #print("fhi={:.3f} betha_cc={:.3f} fhi_0={:.3f}".format(phi, beta_cc, phi_0))
    print("Shrinkage strain (x1000)={:.3f}".format(shrink*1000))

interactive(children=(FloatLogSlider(value=100000.0, description='t (d):', max=6.0, readout_format='.0f', step…

In [21]:
_t = 100000
_h0 = 150
_rh = 80
_ts = 3
_fck = 20
_cem = 'Type N'
shrinkstrain(_t,_h0,_ts,_rh,_fck,_cem)

Shrinkage strain (x1000)=0.551


In [22]:
tt = np.logspace(1.2, 4)
vfunc = np.vectorize(shrink_strain)
@interact(h0=h0, rh=rh, fck=fck, cem=cem)
def shrinkstrain(h0=100, rh=65,  fck=20.0, cem='Type N'):
    shrink = vfunc(tt, h0, 1.0, rh, fck, cem)*1000.0
    print("Total shrinkage strain (x1000) = {:.3f}".format(shrink[49]))
    plt.xscale("linear")
    plt.xlabel("Time (days)")
    plt.ylabel("Total shrinkage")
    plt.plot(tt, shrink)

interactive(children=(FloatSlider(value=200.0, description='h0 (mm):', max=1000.0, min=100.0, readout_format='…